In [19]:
import pandas as pd
import duckdb as ddb

In [20]:
con = ddb.connect("data/ncm-research-data.duckdb")

## Observations Sampling

Choosing a appropriate standard that could distinguish active and inactive users is a challenge. Due to the limitation of sample period, It's not observable whether a user that first appeared near the end of sample period was still active after several weeks. For this reason, we only focused on the users that appeared in the first 7 days, so that we have observation window as long as possible.

This guarantees that, we can trace all these users for at least 21 days. For this reason, our definition of being active is that a user had at least one impression two weeks after his first appearance. Although 14 days are usually not seen as "long-term" in the business context, it ensures that we do not wrongly categorizes users as "inactive" due to sample period limitation. Besides, it balances the proportion of active and inactive users.

With this definition in mind, we then collect these observations and create labels for them.

In [ ]:
con.sql(
    """
CREATE OR REPLACE TABLE main.observation_label AS 
WITH first_act AS (
SELECT
	userId,
	MIN(dt) AS first_act_dt
FROM main.impression i 
GROUP BY 1
HAVING MIN(dt) <= 7
),
act_date AS (
SELECT
	userId,
	dt
FROM main.impression i
GROUP BY 1, 2
)
SELECT 
	fa.userId,
	fa.first_act_dt,
	CASE
		WHEN COUNT(ad.dt) > 0 THEN 0
		ELSE 1
	END AS is_inactive
FROM first_act fa
LEFT JOIN act_date ad ON fa.userId = ad.userId
	AND ad.dt >= fa.first_act_dt + 14 
GROUP BY 1, 2
ORDER BY 1
"""
)

In [ ]:
## Save 7 days impressions for these users
con.sql(
    """
CREATE OR REPLACE TABLE main.observation_impression AS
SELECT 
	i.userId,
	dt,
	EPOCH_MS(impressTime) AS impressTimeStamp,
	impressPosition,
	mlogId,
	isClick,
	isComment,
	isIntoPersonalHomepage,
	isShare,
	isViewComment,
	mlogViewTime,
	detailMlogInfoList 
FROM main.observation_label ol LEFT JOIN main.impression i 
ON ol.userId = i.userId AND i.dt <= ol.first_act_dt + 7
ORDER BY i.userId, impressTime, impressPosition 
"""
)

## Feature Engineering

### Static user demographics data

According to the user level EDA, we believe that several demographical fields could be included into the features for their ability to demonstrate individual differences between users (*age*, *gender*, *province*) and also dependence on the platform (*registeredMonthCnt*, *followCnt*). However, before they can be incorporated with behavioral data, there still exist issues to be solved. Among them, missing data is something that cannot be dismissed. Although all of the fields have missing data, *age* and *gender* are the most serious ones with missing values accounting for 35.2% in our observation users set.

In [ ]:
con.sql(
    """
WITH observations AS (
    SELECT 
        userId,
        MIN(dt) AS first_act_dt
    FROM main.impression
    GROUP BY 1
    HAVING MIN(dt) <= 7
)
SELECT 
    o.userId,
    province,
    age,
    gender,
    registeredMonthCnt,
    followCnt,
    level
FROM observations o 
LEFT JOIN main.user_demographics ud ON o.userId = ud.userId
"""
).to_df().isnull().mean()

userId                0.000000
province              0.000070
age                   0.352695
gender                0.352695
registeredMonthCnt    0.000070
followCnt             0.000070
level                 0.000070
dtype: float64

Since *gender* is a categorical field and the dataset already contains users whose *gender* is marked as "unknown". We decided to follow this practice and substitute null values with "unknown" as well. Regarding that *age* is a continuous numeric field, decision is made to fill null values with median and create a new binary field *is_age_missing* to convey this message in a different way.

For other fields such as *registeredMonthCnt* and *followCnt*, we agreed to fill them with median for its relatively low percentage in missing values, and for *province* we use the value with highest frequency because it is a categorical field.

After the data cleaning for observations' demographics data, we store them into a new table for later use.

In [ ]:
con.sql(
    """
CREATE OR REPLACE TABLE main.observation_demographics AS 
WITH observations AS (
	SELECT
		userId,
		MIN(dt) AS first_act_dt
	FROM
		main.impression
	GROUP BY 1
	HAVING MIN(dt) <= 7
),
mode_province AS (
	SELECT 
		province,
		COUNT(*)
	FROM main.user_demographics
	GROUP BY 1
	ORDER BY COUNT(*) DESC
	LIMIT 1
),
medians AS (
	SELECT
    MEDIAN(age) AS median_age,
    MEDIAN(registeredMonthCnt) AS median_registeredMonthCnt,
    MEDIAN(followCnt) AS median_followCnt,
    MEDIAN(level) AS median_level
FROM main.user_demographics
)
SELECT
	o.userId,
	COALESCE(province, (SELECT province FROM mode_province)) AS province,
	COALESCE(age, (SELECT median_age FROM medians)) AS age,
	COALESCE(gender, 'unknown') AS gender,
	IF(age IS NULL, 1, 0) AS is_age_missing,
	COALESCE(registeredMonthCnt, (SELECT median_registeredMonthCnt FROM medians)) AS registeredMonthCnt,
	COALESCE(followCnt, (SELECT median_followCnt FROM medians)) AS followCnt,
	COALESCE(level, (SELECT median_level FROM medians)) AS level
FROM
	observations o
LEFT JOIN main.user_demographics ud ON o.userId = ud.userId
ORDER BY o.userId
"""
)

### Behavioral data (extracted from impressions)

Traditional machine learning models relies heavily on well-defined empirically extracted features. To supply detailed user profile information to the fullest extent, we categorized features to be considered into 3 types:

- **Aggregated**: A summary to the user's activity intensity during observation window, including mainly total number of each type of actions and several ratio metrics regardless of total active days.

- **Per-day**: Daily averages of each type of actions and ratio metrics per day, used to preserve details about whether a user's usage is steady from day to day.

- **Preference**: Used to reflect a user's preference about mlog and creators, some of them are linked to the external tables such as demographics of mlogs or users.

#### Aggregated

In the following are features that belong to "aggregated":

- *active days*: Total number of days that the user is active (has at least one impression) during the input time range.

- *total_impressions*: Total number of impressions during the input time range

- *total_clicks*: Total number of clicks during the input time range

- *total_likes/total_shares/total_comments/total_into_personal_page/total_view_comments.*: Total number of respective actions

- *total_mlog_watchtime*: Total time a user spent on watching mlogs (in minutes)

- *ctr*: impression Click-Through Rate (*total_clicks* divided by *total_impressions*), representing the percentage of impressions that has triggered a click by a user

- *interaction_rate*: refering to percentage of impressions that triggered at least one type of action besides clicks.

- *like_rate/share_rate/comment_rate*: refering to the percentage of impressions that triggered at least respective type of feedback among **clicked** impressions.

#### Per-day

In the following are the features that belong to "Per-day":

- *avg_daily_impressions*: Average daily impressions for a user

- *avg_daily_clicks*: Avergae daily clicks for a user

- *avg_daily_mlog_watchtime*: Average daily mlog watch time for a user (in minutes)

- *avg_daily_likes/avg_daily_shares/avg_daily_shares*: Average daily amoung of three types of feedback for a user

#### Preference

These are features categorized as "Preference":

- *avg_impression_position*: Avg impression postion for a user during the input time range. Higher number indicates that a user has stronger intention to scroll down.

- *favorite_hour*: Refers to the hour (1-24) that highest frequency for a user, reflecting a user's habit on using the application.

- *favorite_format*: Refers to *type* in mlog_demographics with higher frequency among **clicked** impressions, reflecting a user's preference over music video or images with background music.

- *favorite_creator_type*: Refers to *creatorType* in creator_demographics with highest frequency among impressions triggered **like/share/comment**, since we considered these three type of actions reflect a user's positive feedback to the creator.

- *avg_user_likes_in_clicked/avg_user_shares_in_clicked/avg_user_comments_in_clicked*: Refers to average number of *userLikeCount/userShareCount/userCommentCount* in mlog_stats among **clicked** impressions. These features reflects a user's intention to click into content which is already liked/shared/commented by many users among the Cloud Village community.

- *avg_creator_followers_in_feedbacks*: Refers to average number of *followeds* in creator_demographics among impressions triggered **like/share/comment**. This reflects a user's intention to interact with popular and well-known creators.

In [ ]:
## （2025/10/10） Final SQL to be finished
con.sql(
    """"""
)

In [ ]:
con.close()